<a href="https://colab.research.google.com/github/vaibhavch19/Movie-Recommendation-System/blob/main/Movie_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the dependencies

In [7]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Data Collection and Pre-Processing

In [13]:
# loading the data from the csv file to apandas dataframe
# Specify encoding explicitly when reading the CSV file
movies_data = pd.read_csv('/content/movies.csv', encoding='utf-8', skiprows=lambda x: x == 2903)

In [15]:
# printing the first 5 rows of the dataframe
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [16]:
# number of rows and columns in the data frame

movies_data.shape

(2902, 24)

In [17]:
# selecting the relevant features for recommendation

selected_features = ['genres','keywords','tagline','cast','director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [18]:
# replacing the null valuess with null string

for feature in selected_features:
  movies_data[feature] = movies_data[feature].fillna('')

In [19]:
# combining all the 5 selected features

combined_features = movies_data['genres']+' '+movies_data['keywords']+' '+movies_data['tagline']+' '+movies_data['cast']+' '+movies_data['director']

In [20]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
2897    Thriller Science Fiction Mystery double life u...
2898    Drama History War war crimes mass murder war v...
2899    Animation Family Action Adventure Comedy  From...
2900    Science Fiction   Hiroyuki Ikeuchi Takuya Kimu...
2901    War Drama journalist interpreter georgia europ...
Length: 2902, dtype: object


In [21]:
# converting the text data to feature vectors

vectorizer = TfidfVectorizer()

In [22]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [23]:
print(feature_vectors)

  (0, 1611)	0.17221894784568656
  (0, 5263)	0.11644439372909768
  (0, 8814)	0.1921204506213619
  (0, 6909)	0.16030109297432038
  (0, 5922)	0.23174035356540162
  (0, 9890)	0.1569607365680705
  (0, 11304)	0.1962416668526501
  (0, 9511)	0.20266389384950995
  (0, 9016)	0.21292914649227862
  (0, 11742)	0.20844055497721534
  (0, 11557)	0.22778113525669846
  (0, 9039)	0.14964206967627752
  (0, 7761)	0.26471044229332946
  (0, 7557)	0.09122702393845371
  (0, 11553)	0.12439048522579056
  (0, 10329)	0.06967511167005394
  (0, 3324)	0.2414106437051432
  (0, 9654)	0.2154202492095482
  (0, 2146)	0.24753929238283384
  (0, 11240)	0.12353943967114835
  (0, 9729)	0.33029994402013
  (0, 3947)	0.16457535111363558
  (0, 2044)	0.2362289450804649
  (0, 2465)	0.2414106437051432
  (0, 3663)	0.10074767851972181
  :	:
  (2901, 5130)	0.26502202044479917
  (2901, 1577)	0.24283706395062077
  (2901, 3395)	0.22985969632139613
  (2901, 11746)	0.2520446528155745
  (2901, 1981)	0.2520446528155745
  (2901, 3281)	0.2168823

Cosine Similarity

In [24]:
# getting the similarity scores using cosine similarity

similarity = cosine_similarity(feature_vectors)

In [25]:
print(similarity)

[[1.         0.06693627 0.03576518 ... 0.014322   0.01937822 0.03368582]
 [0.06693627 1.         0.02982348 ... 0.0196509  0.         0.01219828]
 [0.03576518 0.02982348 1.         ... 0.01489333 0.         0.        ]
 ...
 [0.014322   0.0196509  0.01489333 ... 1.         0.         0.        ]
 [0.01937822 0.         0.         ... 0.         1.         0.        ]
 [0.03368582 0.01219828 0.         ... 0.         0.         1.        ]]


In [26]:
print(similarity.shape)

(2902, 2902)


Getting the movie name from the user

In [27]:
# getting the movie name from the user

movie_name = input(' Enter your favourite movie name : ')

 Enter your favourite movie name : toy story


In [28]:
# creating a list with all the movie names given in the dataset

list_of_all_titles = movies_data['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [29]:
# finding the close match for the movie name given by the user

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['Toy Story', 'Toy Story 3', 'Toy Story 2']


In [30]:
close_match = find_close_match[0]
print(close_match)

Toy Story


In [31]:
# finding the index of the movie with title

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
print(index_of_the_movie)

1541


In [32]:
# getting a list of similar movies

similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, 0.0), (1, 0.0), (2, 0.0), (3, 0.023313015683685715), (4, 0.0), (5, 0.0), (6, 0.029545383124903474), (7, 0.0), (8, 0.032612743174230394), (9, 0.0), (10, 0.0), (11, 0.0), (12, 0.0), (13, 0.0), (14, 0.0), (15, 0.010232213826183653), (16, 0.0), (17, 0.0), (18, 0.0053559108676013096), (19, 0.0), (20, 0.0), (21, 0.0), (22, 0.0), (23, 0.0), (24, 0.03895580137506648), (25, 0.0), (26, 0.0), (27, 0.0), (28, 0.0), (29, 0.0), (30, 0.04766043717684022), (31, 0.02890005136931269), (32, 0.03588485717418231), (33, 0.0), (34, 0.06189556191030347), (35, 0.0), (36, 0.0), (37, 0.011248054911584836), (38, 0.0), (39, 0.0), (40, 0.1515999719535767), (41, 0.027038837463528156), (42, 0.30013368208301167), (43, 0.0), (44, 0.0), (45, 0.0), (46, 0.0), (47, 0.036677027815802565), (48, 0.010980187284494172), (49, 0.0), (50, 0.0), (51, 0.0), (52, 0.011668515686188248), (53, 0.03655116219755853), (54, 0.06339576168726437), (55, 0.03641754065287196), (56, 0.0), (57, 0.05271785485985009), (58, 0.006460203296720796

In [33]:
len(similarity_score)

2902

In [34]:
# sorting the movies based on their similarity score

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
print(sorted_similar_movies)

[(1541, 1.0), (343, 0.3615430722334525), (42, 0.30013368208301167), (40, 0.1515999719535767), (301, 0.14774315229034174), (184, 0.14055832526201611), (1612, 0.13304822187691728), (2314, 0.1294587484952345), (2189, 0.12921983804330753), (1246, 0.12816419449759953), (2500, 0.1258518075453641), (727, 0.12580487744380123), (2771, 0.12462933603383852), (2761, 0.11380707328916814), (2501, 0.11121105461576253), (566, 0.1095497299192565), (752, 0.1085855812646465), (1191, 0.1073080274466366), (1386, 0.10646365641904788), (1426, 0.10506931065526004), (1312, 0.1027802085710054), (1257, 0.10134039905187692), (2876, 0.10116488373235653), (892, 0.1001866402821995), (533, 0.09733392100851848), (742, 0.09363081540885812), (1795, 0.09219858525961219), (90, 0.09109338920805261), (820, 0.08935981284933187), (903, 0.08838991375096403), (1856, 0.08757099219365824), (2239, 0.08743938810126156), (1053, 0.08664798268499534), (927, 0.0861554067255661), (2846, 0.08513657597852098), (1384, 0.08475184396684834),

In [35]:
# print the name of similar movies based on the index

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

Movies suggested for you : 

1 . Toy Story
2 . Toy Story 2
3 . Toy Story 3
4 . Cars 2
5 . Cloud Atlas
6 . Home
7 . Larry Crowne
8 . Big
9 . A Guy Thing
10 . Quest for Camelot
11 . Vamps
12 . Jingle All the Way
13 . Admission
14 . Dirty Work
15 . Hachi: A Dog's Tale
16 . Cars
17 . My Favorite Martian
18 . Small Soldiers
19 . Saving Mr. Banks
20 . Valiant
21 . Joe Somebody
22 . Fantastic Mr. Fox
23 . Last Orders
24 . Casino
25 . Monster House
26 . The Boxtrolls
27 . Silverado
28 . The Polar Express
29 . Casper


Movie Recommendation Sytem

In [36]:
movie_name = input(' Enter your favourite movie name : ')

list_of_all_titles = movies_data['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

 Enter your favourite movie name : inception
Movies suggested for you : 

1 . Inception
2 . Treasure Planet
3 . The Dark Knight Rises
4 . The Last Samurai
5 . In Dreams
6 . Looper
7 . Highlander: Endgame
8 . G.I. Joe: The Rise of Cobra
9 . The Revenant
10 . Cold Mountain
11 . The Walk
12 . Sin City: A Dame to Kill For
13 . The Count of Monte Cristo
14 . Catch Me If You Can
15 . Solaris
16 . The Man in the Iron Mask
17 . The Juror
18 . Top Gun
19 . Eternal Sunshine of the Spotless Mind
20 . Blood Diamond
21 . Premium Rush
22 . Letters from Iwo Jima
23 . The Silence of the Lambs
24 . RockNRolla
25 . Miss Potter
26 . Lincoln
27 . Without a Paddle
28 . Shutter Island
29 . Whip It
